In [1]:
!pip install langchain_community
!pip install langchain_openai
!pip install panel
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling op

In [7]:
import os
import json
from typing import List
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from pydantic import BaseModel, Field
import panel as pn
from dotenv import load_dotenv
import openai
from langchain_openai import ChatOpenAI
from langchain.memory.chat_message_histories import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.memory import ChatMessageHistory
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
import time

# Set up logging
import logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv()

# Ensure Panel extensions are loaded
pn.extension()

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = "Enter API Key Here"
openai.api_key = os.environ['OPENAI_API_KEY']

In [15]:
class WoundCareInputs(BaseModel):
    location: str = Field(..., description="The location of the wound (e.g., hands, legs, arms, head).")
    severity: str = Field(..., description="The severity of the wound (e.g., minor, moderate, severe).")


def wc_instructions(location: str, severity: str) -> List[str]:
    loc = location.lower()
    sev = severity.lower()
    if loc in wound_care_guide and sev in wound_care_guide[loc]:
        return wound_care_guide[loc][sev]
    else:
        return ["Invalid location or severity provided."]

@tool
def get_woundcare(location: str, severity: str) -> str:
    """Provide essential wound care instructions based on location and severity."""
    wound_care_guide = {
        "hands": {
            "minor": [
                "Clean the wound with mild soap and water.",
                "Apply an antiseptic ointment.",
                "Cover with a sterile bandage.",
                "Change the bandage daily."
            ],
            "moderate": [
                "Clean the wound thoroughly with sterile saline solution or clean water.",
                "Apply pressure with sterile gauze to stop bleeding if necessary.",
                "Use an antiseptic ointment.",
                "Cover with a sterile bandage and change it twice daily.",
                "Monitor for signs of infection (redness, swelling, pain)."
            ],
            "severe": [
                "Apply pressure immediately to stop the bleeding with a clean cloth or sterile gauze.",
                "Seek medical help as soon as possible.",
                "If possible, keep the wound elevated.",
                "Do not remove any embedded objects; allow a healthcare professional to handle this."
            ]
        }
    }
    instructions = wound_care_guide.get(location.lower(), {}).get(severity.lower(), ["Invalid input"])
    return "\n".join(instructions)

# Initialize the agent
def initialize_wound_care_agent():
    tools = [get_woundcare]

    llm = ChatOpenAI(temperature=0.3, openai_api_key=os.getenv("OPENAI_API_KEY"))

    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    return agent





In [4]:
tools = [get_woundcare]



In [16]:
def parse_inputs(query: str) -> dict:
    """Parses the user query to extract location and severity."""
    # Default values for location and severity
    location = severity = None

    # Define keywords for locations and severities
    locations = ['hands', 'legs', 'arms', 'head']
    severities = ['minor', 'moderate', 'severe']

    # Split the query into words
    words = query.lower().split()

    # Match location and severity from the query
    for word in words:
        if word in locations:
            location = word
        if word in severities:
            severity = word

    # Return parsed inputs
    return {"location": location, "severity": severity}

# Updated memory initialization
class WoundCareAssistant:
    def __init__(self, tools, **params):
        super(WoundCareAssistant, self).__init__(**params)
        self.panels = []  # Chat history panels

        # Convert tools to OpenAI function-compatible format
        self.functions = [format_tool_to_openai_function(f) for f in tools]

        # Bind tools to the model
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)

        # Set up memory
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

        # Define the agent's prompt
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful but sassy assistant."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        # Set up the chain
        self.chain = (
            RunnablePassthrough.assign(
                agent_scratchpad=lambda x: format_to_openai_functions(x.get("intermediate_steps", []))
            ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        )

        # Agent executor
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    def convchain(self, query):

        if not query:
            return

        try:

            # Run the query through the agent
            result = self.qa.invoke({"input": query})
            self.answer = result["output"]
        except Exception as e:
            self.answer = f"An error occurred: {e}"

        # Update the chat panels
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])

        # Return the updated chat widget
        return pn.Column(*self.panels)

# Initialize the assistant
assistant = WoundCareAssistant(tools=tools)


In [17]:

# UI
inp = pn.widgets.TextInput(name='Ask the Assistant', placeholder='Type your question here...')
send_button = pn.widgets.Button(name='Send', button_type='primary')
chat_display = pn.Column()

def update_chat(event):
    if inp.value:
        response = assistant.convchain(inp.value)
        chat_display.objects = [response]
        inp.value = ""  # Clear the input field

send_button.on_click(update_chat)

dashboard = pn.Column(
    pn.pane.Markdown("### Wound Care Assistant"),
    inp,
    pn.Row(send_button),
    chat_display
)

dashboard

Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 477, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 453, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 244, in handle_event
 event_cls._handle_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 279, in _handle_event
 cb(event.msg_data)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 400, in trigger_event
 model._trigger_event(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 111, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 262, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 453, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 107, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 558, in _comm_event
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 468, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 556, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 493, in _process_bokeh_event
 self._process_event(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/widgets/button.py", line 242, in _process_event
 self.clicks += 1
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 515, in _f
 instance_param.__set__(obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 517, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameters.py", line 541, in __set__
 super().__set__(obj,val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 517, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1564, in __set__
 obj.param._call_watcher(watcher, event)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2604, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2586, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "<ipython-input-17-1a2a38b48ab7>", line 8, in update_chat
 response = assistant.convchain(inp.value)
 File "<ipython-input-16-44cb78ee4e2f>", line 57, in convchain
 time_langchain_agent_response(self.qa,query)
 File "<ipython-input-15-2efee7492d47>", line 60, in time_langchain_agent_response
 response = agent.run(query) # Generate response using LangChain agent
 File "/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py", line 182, in warning_emitting_wrapper
 return wrapped(*args, **kwargs)
 File "/usr/local/lib/pytho

Column
    [0] Markdown(str)
    [1] TextInput(name='Ask the Assistant', placeholder='Type your question h...)
    [2] Row
        [0] Button(button_type='primary', name='Send')
    [3] Column()

<ipython-input-15-2efee7492d47>:60: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)  # Generate response using LangChain agent
